# Astro Lab Book 2025

In [1]:
import astropy
import numpy as np
import ccdproc as ccdp
from pathlib import Path
from astropy.nddata import CCDData
from astropy.stats import mad_std
import astropy.units as u
import matplotlib.pyplot as plt
import warnings
import glob
import os
import photutils.background as photobg
import photutils.segmentation as photoseg
from astropy.convolution import convolve
from photutils.segmentation import SourceCatalog
from photutils.utils import calc_total_error
import photutils.aperture as photoap
from photutils.background import MedianBackground
from scipy.optimize import curve_fit

warnings.filterwarnings(action='once') #prevents warnings repeating and clogging workspace

In [2]:
biasImg = astropy.nddata.CCDData.read('data/bias/20181101.bias.00000002.fits', unit='adu')
biasimg = np.asarray(biasImg, dtype=np.float64)

def biasavg(biasframe):
    total = 0
    x = len(biasframe)
    y = len(biasframe[0]) #as the array is a square this returns the length of all columns

    for i in range(x):
        for j in range(y):
            total+=biasframe[i][j]

    biasavg = total/(x*y) #mean

    myvar = 0 #filler variable

    for i in range(x):
        for j in range(y):
            myvar+=((biasframe[i][j] - biasavg)**2)

    biasdev = np.sqrt(myvar/(x*y)) #standard deviation

    output = [biasavg, biasdev]
    
    return(output)

biasavg1 = biasavg(biasimg)

print(biasavg1[0])
print(biasavg1[1])

960.1504330039024
8.422060711910369


In [3]:
warnings.filterwarnings('ignore') #this otherwise outputs many, many warnings

calibrated_path_bias = Path('data/bias')
#allbias = ccdp.ImageFileCollection(calibrated_path_bias)
calibrated_biases = glob.glob("data/bias/*.fits")

#calibrated_biases = allbias.files_filtered(imagetyp='Bias Frame', include_path=True)

combined_bias = ccdp.combine(calibrated_biases, #combines all bias frames into my master bias frame
                             method='average',
                             sigma_clip=True, sigma_clip_low_thresh=5, sigma_clip_high_thresh=5, 
                             sigma_clip_func=np.ma.median, sigma_clip_dev_func=mad_std, #the sigma_clip removes any significant outliers from the data
                             mem_limit=350e6,
                             unit='adu'
                            )

combined_bias.meta['combined'] = True

INFO: splitting each image into 11 chunks to limit memory usage to 350000000.0 bytes. [ccdproc.combiner]


In [4]:
masterbias = np.asarray(combined_bias)

bias = biasavg(masterbias)[0]
read_noise = biasavg(masterbias)[1]

print(bias) #bias
print(read_noise) #read noise

962.9592900923088
5.350563223315935


In [5]:
biasnum = len(calibrated_biases)
biasdev1 = biasavg(biasimg)[1]
biasdevmaster = biasavg(masterbias)[1]

expecteddev = biasdev1 / np.sqrt(biasnum) #what our expected relation gives

print((biasdevmaster - expecteddev)/expecteddev)

0.4205814342237132


In [6]:
#uncalibrated_path_dark = Path('data/dark')
#calibrated_path_dark = Path('data/calibrated_darks')

#alldark = ccdp.ImageFileCollection(uncalibrated_path_dark)
alldark = glob.glob("data/darks/*.fits")

for item in alldark:
    ccd = astropy.nddata.CCDData.read(item, unit='adu')
    ccd = ccdp.subtract_bias(ccd, combined_bias) #subtracts bias from each dark frame
    file_name = os.path.basename(item)
    ccd.write(Path('data/calibrated_darks') / file_name, overwrite=True) #writes calibrated dark into new folder

In [7]:
#allcalibrateddark = ccdp.ImageFileCollection('data/calibrated_darks')
calibrated_darks = glob.glob("data/calibrated_darks/*.fits")

#calibrated_darks = allcalibrateddark.files_filtered(imagetyp='Dark Frame', include_path=True)

combined_dark = ccdp.combine(calibrated_darks, #combines all dark frames into my master dark frame
                             method='median',  #uses median avg rather than mean                          
                             mem_limit=350e6,
                             unit='adu'
                            )

combined_dark.meta['combined'] = True

INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: splitting each image into 23 chunks to limit memory usage to 350000000.0 bytes. [ccdproc.combiner]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the uni

In [8]:
dark = np.asarray(combined_dark) #gets the master dark

pixtotal = 0

x = len(dark)
y = len(dark[0]) #as the array is a square this returns the length of all columns

for i in range(x):
    for j in range(y):
        pixtotal+=dark[i][j]

darkcurrentavg = 2.2 * pixtotal/(x*y)

print(f"Dark Current Avg: {darkcurrentavg} e/pix/s") 

Dark Current Avg: 9.393139339485636 e/pix/s


In [11]:
flats_B = glob.glob("data/flat_B/*.flat.*.fits")
flats_R = glob.glob("data/flat_R/*.flat.*.fits")

for item in flats_B:
    ccd = astropy.nddata.CCDData.read(item, unit='adu')
    ccdp.subtract_bias(ccd, combined_bias)
    file_name = os.path.basename(item)
    ccd.write(Path("data/flat_B_semi-cali") / file_name, overwrite=True)

for item in flats_R:
    ccd = astropy.nddata.CCDData.read(item, unit='adu')
    ccdp.subtract_bias(ccd, combined_bias)
    file_name = os.path.basename(item)
    ccd.write(Path("data/flat_R_semi-cali") / file_name, overwrite=True)

In [13]:
allflats_B = glob.glob("data/flat_B_semi-cali/*.flat.*.fits")
allflats_R = glob.glob("data/flat_R_semi-cali/*.flat.*.fits")

for item in allflats_B:
    ccd = astropy.nddata.CCDData.read(item, unit='adu')
    ccd = ccdp.subtract_dark(ccd, combined_dark, exposure_time="EXPTIME", exposure_unit=u.second) #CHECK EXPOSURE TIMES
    file_name = os.path.basename(item)
    ccd.write(Path("data/flat_B_cali") / file_name, overwrite=True) #writes calibrated flat frame to new folder

for item in allflats_R:
    ccd = astropy.nddata.CCDData.read(item, unit='adu')
    ccd = ccdp.subtract_dark(ccd, combined_dark, exposure_time="EXPTIME", exposure_unit=u.second) #CHECK EXPOSURE TIMES
    file_name = os.path.basename(item)
    ccd.write(Path("data/flat_R_cali") / file_name, overwrite=True) #writes calibrated flat frame to new folder

INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader inste

In [14]:
allcalflats_B = glob.glob("data/flat_B_cali/*.flat.*.fits")
allcalflats_R = glob.glob("data/flat_R_cali/*.flat.*.fits")

for item in allcalflats_B:
    ccd = astropy.nddata.CCDData.read(item, unit='adu')
    
    total = 0
    mean = 0
    ccdarray = np.asarray(ccd)
    
    for i in range(len(ccdarray)):
        for j in range(len(ccdarray[1])):
            total += ccdarray[i, j]
    
    mean = total / (i*j)

    for i in range(len(ccdarray)):
        for j in range(len(ccdarray[1])):
            ccdarray[i, j] /= mean
    
    ccd = CCDData(ccdarray, unit="adu")
    file_name = os.path.basename(item)
    ccd.write(Path("data/flat_B_norm") / file_name, overwrite=True) #writes normalised flat frame to new folder

for item in allcalflats_R:
    ccd = astropy.nddata.CCDData.read(item, unit='adu')
    
    total = 0
    mean = 0
    ccdarray = np.asarray(ccd)
    
    for i in range(len(ccdarray)):
        for j in range(len(ccdarray[1])):
            total += ccdarray[i, j]
    
    mean = total / (i*j)

    for i in range(len(ccdarray)):
        for j in range(len(ccdarray[1])):
            ccdarray[i, j] /= mean
    
    ccd = CCDData(ccdarray, unit="adu")
    file_name = os.path.basename(item)
    ccd.write(Path("data/flat_R_norm") / file_name, overwrite=True) #writes normalised flat frame to new folder

INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader inste

In [15]:
calibrated_flats_R = glob.glob("data/flat_R_norm/*.flat.*.fits")
calibrated_flats_B = glob.glob("data/flat_B_norm/*.flat.*.fits")

combined_flat_R = ccdp.combine(calibrated_flats_R, #combines all normalised flat frames into my master flat frame
                             method='median',  #uses median avg rather than mean                          
                             mem_limit=350e6,
                             unit='adu'
                            )

combined_flat_B = ccdp.combine(calibrated_flats_B, #combines all normalised flat frames into my master flat frame
                             method='median',  #uses median avg rather than mean                          
                             mem_limit=350e6,
                             unit='adu'
                            )

combined_flat_R.meta['combined'] = True
combined_flat_B.meta['combined'] = True

INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: splitting each image into 16 chunks to limit memory usage to 350000000.0 bytes. [ccdproc.combiner]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the uni

In [18]:
science_R = glob.glob('data/science_R/*.fits')
science_B = glob.glob('data/science_B/*.fits') 

for frame in science_R:
    
    ccd = astropy.nddata.CCDData.read(frame, unit="adu")
    ccd = ccdp.subtract_bias(ccd, combined_bias) #subtracts master bias from each science frame  
    ccd = ccdp.subtract_dark(ccd, combined_dark, exposure_time='EXPTIME', exposure_unit=u.second, scale=True) #subtracts master dark from each science frame
    file_name = os.path.basename(frame)
    ccd.write(Path("data/science_R_semi-cali") / file_name, overwrite=True) #writes calibrated science frame to new folder

for frame in science_B:
    
    ccd = astropy.nddata.CCDData.read(frame, unit="adu")
    ccd = ccdp.subtract_bias(ccd, combined_bias) #subtracts master bias from each science frame  
    ccd = ccdp.subtract_dark(ccd, combined_dark, exposure_time='EXPTIME', exposure_unit=u.second, scale=True) #subtracts master dark from each science frame
    file_name = os.path.basename(frame)
    ccd.write(Path("data/science_B_semi-cali") / file_name, overwrite=True) #writes calibrated science frame to new folder

In [ ]:
science_reduced_R = Path('data/science_R_semi-cali')
science_reduced_B = Path('data/science_B_semi-cali')

semicalibratedscience = glob.glob('obslab-compskills/obslab/raw/semicalibratedscience/*.fits') 

for frame in semicalibratedscience:
    
    ccd = astropy.nddata.CCDData.read(frame, unit="adu")
    ccdp.ccd_process(ccd, master_flat=combined_flat)
    ccdarray = np.asarray(ccd)
    
    for i in range(len(ccdarray)):
        for j in range(len(ccdarray[1])):
            ccdarray[i, j] *= 2 #doubles exposure time by doubling value in each pixel
    
    ccd = CCDData(ccdarray, unit="adu")
    
    ccd.write(Path("") / f'{filenum}.fits', overwrite=False) #writes calibrated science frame to new folder